### Import libraries and get response code

In [1]:
import requests
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup

In [2]:
# Send a request to the page
url = 'https://www.reed.co.uk/jobs/data-analyst-jobs'
response = requests.get(url)

In [3]:
# Download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

200

### Scrape job listings (reed.co.uk)

In [4]:
# Create an empty list
jobs = []

# Loop through all pages
for page in range(1, 33): 
    url = f'https://www.reed.co.uk/jobs/data-analyst-jobs-in-united-kingdom?pageno={page}&orgId=1&direct=True'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract job details
    job_listings = soup.select('div.row > div.col-xs-12.col-sm-8.col-md-9.details')
    for job in job_listings:
        # Extract the job title
        title = job.find('h2').text.strip()

        # Extract the company name
        company = job.find('a', {'class': 'gtmJobListingPostedBy'}).text.strip()

        # Extract the location (if available)
        location_elem = job.find('li', {'class': 'job-metadata__item--location'})
        location = location_elem.text.strip() if location_elem else 'Not specified'

        # Extract the salary (if available)
        salary_elem = job.find('li', {'class': 'job-metadata__item--salary'})
        salary = salary_elem.text.strip() if salary_elem else 'Not specified'

        # Extract job type
        jobtype = job.find('li', {'class': 'job-metadata__item--type'}).text.strip()

        # Extract job pref
        remote_elem = job.find('li', {'class': 'job-metadata__item--remote'})
        remote = remote_elem.text.strip() if remote_elem else 'Not specified'

        # Extract the job description
        job_link = 'https://www.reed.co.uk' + job.find('a', href=True)['href']
        job_response = requests.get(job_link)
        job_soup = BeautifulSoup(job_response.content, 'html.parser')
        job_description = job_soup.find('span', {'itemprop': 'description'})
        if job_description:
            job_description = job_description.text.strip()
        else:
            job_description = 'No description available'
            
        # Append the job information
        jobs.append([title, company, location, salary, jobtype, remote, job_description, job_link])
        

#### Web Crawl 1 Results

In [6]:
#Create a dataframe from the list of jobs
columns = ['title','company','location','salary','type','remote','description','link']
jobs_df = pd.DataFrame(jobs, columns=columns)

In [7]:
jobs_df.shape

(4293, 8)

In [8]:
jobs_df

,title,company,location,salary,type,remote,description,link
0,Data Analyst,e.surv Chartered Surveyors,Kettering\r\n Northamptonshire,Salary negotiable,"Permanent, full-time",Work from home,"Trading since 1989, e.surv Chartered Surveyors...",https://www.reed.co.uk/jobs/data-analyst/49988...
1,Data Analyst,University of the West of Scotland,Paisley\r\n Renfrewshire,"£32,000 - £39,000 per annum","Contract, full-time",Not specified,"Registry Full time, Fixed-term for 24 months T...",https://www.reed.co.uk/jobs/data-analyst/50197...
2,Data Analyst,Brandon James Ltd,Bristol\r\n Avon,"£40,000 - £45,000 per annum","Contract, full-time",Not specified,A strategic medium sized consultancy is now se...,https://www.reed.co.uk/jobs/data-analyst/50020...
3,Data Analyst,Brandon James Ltd,London\r\n London,"£40,000 - £45,000 per annum","Contract, full-time",Not specified,A strategic medium sized consultancy is now se...,https://www.reed.co.uk/jobs/data-analyst/50020...
4,Data Analyst,The Consultancy Group,London\r\n London,"£35,000 - £50,000 per annum","Permanent, full-time",Not specified,I am partnered with a renowned Retail business...,https://www.reed.co.uk/jobs/data-analyst/50188...
...,...,...,...,...,...,...,...,...
4288,Cloud Platform Engineer,Hays Specialist Recruitment Limited,Ireland,"£40,000 - £50,000 per annum","Permanent, full-time",Not specified,Cloud Platform Engineer - Up to £50k - Hybrid ...,https://www.reed.co.uk/jobs/cloud-platform-eng...
4289,Quantitative Risk Management Analyst Consultant,eFinancialCareers,London\r\n London,Competitive salary,"Permanent, full-time",Not specified,DescriptionJob Description Assist the team in...,https://www.reed.co.uk/jobs/quantitative-risk-...
4290,Test Lead,Proactive Appointments,Windsor\r\n Berkshire,"£42,000 - £52,000 per annum","Permanent, full-time",Not specified,Test Lead Our client is seeking an experienc...,https://www.reed.co.uk/jobs/test-lead/50039066...
4291,Senior Team Lead Quant Analyst/ London,eFinancialCareers,London\r\n London,Competitive salary,"Permanent, full-time",Not specified,Role :- Your role will be to manage a small te...,https://www.reed.co.uk/jobs/senior-team-lead-q...


#### Web Crawl 2 Results

In [6]:
# Create a dataframe from the list of jobs
columns = ['title','company','location','salary','type','remote','description','link']
jobs_df2 = pd.DataFrame(jobs, columns=columns)

In [7]:
jobs_df2.shape

(3753, 8)

In [8]:
jobs_df2

,title,company,location,salary,type,remote,description,link
0,Data Analyst,Surrey County Council,Reigate\r\n Surrey,"£36,989 - £40,146 per annum","Contract, full-time",Not specified,"The starting salary for this role is £36,989 p...",https://www.reed.co.uk/jobs/data-analyst/50512...
1,Data Analyst,UBT,Ipswich\r\n Suffolk,"£38,000 - £40,000 per annum","Permanent, full-time",Not specified,We are currently seeking an experienced Data A...,https://www.reed.co.uk/jobs/data-analyst/50543...
2,Data Analyst,Robert Half,Coventry\r\n West Midlands ...,"£55,000 - £60,000 per annum","Permanent, full-time",Not specified,Robert Half are currently searching for a Data...,https://www.reed.co.uk/jobs/data-analyst/50426...
3,Data Analyst,BeTechnology Group Limited,"Caythorpe, Nottingham\r\n N...","£28,000 - £40,000 per annum","Permanent, full-time",Not specified,Job Title: Junior Data Analyst & Data Analyst ...,https://www.reed.co.uk/jobs/data-analyst/50406...
4,Data Analyst,INTEC SELECT LIMITED,East Croydon\r\n Surrey,"£40,000 - £45,000 per annum","Permanent, full-time",Not specified,"Data Analyst - FTC £40,000 - £45,000 - 12 Mont...",https://www.reed.co.uk/jobs/data-analyst/50460...
...,...,...,...,...,...,...,...,...
3748,Financial Planning & Analysis Manager,Michael Page Finance,Sheffield\r\n South Yorkshire,"£65,000 - £75,000 per annum","Permanent, full-time",Not specified,This role owns the Planning and Performance fo...,https://www.reed.co.uk/jobs/financial-planning...
3749,Risk Manager,Harnham - Data & Analytics Recruitment,Milton Keynes\r\n Buckingha...,"£70,000 - £85,000 per annum","Permanent, full-time",Not specified,"Risk Manager Milton Keynes, hybrid working Up...",https://www.reed.co.uk/jobs/risk-manager/50356...
3750,Associate Project Manager,Hays Specialist Recruitment Limited,Cambridge\r\n Cambridgeshire,£450.00 per day,"Contract, full-time",Not specified,Associate Project Manager Overview: In 2023 we...,https://www.reed.co.uk/jobs/associate-project-...
3751,Finance Analyst,Elevation Recruitment Group,Brigg\r\n South Humberside,"£26,000 - £42,000 per annum","Permanent, full-time",Not specified,Elevation Accountancy & Finance are delighted ...,https://www.reed.co.uk/jobs/finance-analyst/50...


#### Web Crawl 3 Results

In [5]:
# Create a dataframe from the list of jobs
columns = ['title','company','location','salary','type','remote','description','link']
jobs_df3 = pd.DataFrame(jobs, columns=columns)

In [6]:
jobs_df3.shape

(864, 8)

In [7]:
jobs_df3

,title,company,location,salary,type,remote,description,link
0,Data Analyst,Surrey County Council,Reigate\r\n Surrey,"£36,989 - £40,146 per annum","Contract, full-time",Not specified,"The starting salary for this role is £36,989 p...",https://www.reed.co.uk/jobs/data-analyst/50512...
1,Data Analyst,UBT,Ipswich\r\n Suffolk,"£38,000 - £40,000 per annum","Permanent, full-time",Not specified,We are currently seeking an experienced Data A...,https://www.reed.co.uk/jobs/data-analyst/50543...
2,Data Analyst,Reed,Edinburgh\r\n Midlothian,"£200.00 - £240.00 per day, inc benefits","Contract, full-time",Not specified,Are you a data analyst looking for your next c...,https://www.reed.co.uk/jobs/data-analyst/50578...
3,Data Analyst,eFinancialCareers,Belfast\r\n County Antrim,Competitive salary,"Permanent, full-time",Not specified,Job Summary Belfast Contract JN -062023-19394...,https://www.reed.co.uk/jobs/data-analyst/50597...
4,Data Analyst,Reed,Binley Industrial Estate\r\n ...,"£30,000 - £35,000 per annum, negotiable","Permanent, full-time",Not specified,Looking for your next exciting opportunity as ...,https://www.reed.co.uk/jobs/data-analyst/50612...
...,...,...,...,...,...,...,...,...
859,Business Analyst - Workflow & Process Engineering,eFinancialCareers,London\r\n London,Competitive salary,"Permanent, full-time",Not specified,"At Bloomberg, our products are fuelled by powe...",https://www.reed.co.uk/jobs/business-analyst-w...
860,Technical Support Assistant,Talent Finder,Corby\r\n Northamptonshire,"£20,000 - £28,000 per annum","Permanent, full-time",Not specified,"Technical Support Assistant | Corby, Northants...",https://www.reed.co.uk/jobs/technical-support-...
861,"Analyst, Software Engineering - AI Labs",eFinancialCareers,Edinburgh\r\n Midlothian,Competitive salary,"Permanent, full-time",Work from home,DescriptionAbout this roleAbout this roleThe A...,https://www.reed.co.uk/jobs/analyst-software-e...
862,Senior Information Security Analyst,AJ Bell,Manchester\r\n Lancashire,"£50,000 - £65,000 per annum","Permanent, full-time",Work from home,To support the IT Risk & Security Manager in m...,https://www.reed.co.uk/jobs/senior-information...


#### Export

In [8]:
character_to_delete = '\ud83c'

# Iterate over each column and check for the character
filtered_df = jobs_df3[~jobs_df3.apply(lambda x: x.astype(str).str.contains(character_to_delete, na=False).any(), axis=1)]


In [9]:
filtered_df.shape

(864, 8)

In [10]:
jobs_df3 = filtered_df

In [11]:
jobs_df3.to_csv("jobs3.csv", index=False)